# Spinning Balls
## Lecture 6

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive

Let's move on from cannon shells to a more friendly game of baseball.  If we want to understand the dynamics of a ball thrown by a pitcher, the approach is still use projectile motion like we did in the previous lecture.

The air drag turns out to be again critically important for correctly predicting the trajectory and range of baseball.  Air drag depends on speed in two ways.  The first is directly, as in

$$ \frac{F_{drag}}{m} = \frac{1}{2} \rho C_d A v^2/m = C_D v^2$$

where the drag force is proportional to the square of the velocity (as well as the cross-sectional area $A$ and air density $\rho$. As before, we can also embed the medium (air) and the geometry (spherical) into a single coefficient, $C_D$.

### Velocity dependent drag

The second is that the coefficient $C_d$ itself is also a function of $v$.

![Drag Coefficient for Baseball](http://farside.ph.utexas.edu/teaching/329/lectures/img362.png)

This plot is taken from a book called *The Physics of Baseball* and appears to be based on experimental data.  The drop in drag coefficient is due to a transition from a laminar flow around the ball to a turbulent flow around the ball.


The textbook by Giordano offers the following approximation for the drag of a normal baseball (in SI units):

$$ C_D = 0.0039 + \frac{0.0058}{1+\exp[(v-v_d)/\Delta]}$$

where $v_d$ = 35 m/s and $\Delta$ = 5 m/s. We call this kind of approximate formula a *parametrization*.  Note that the details of factors such as air density, mass, cross-sectional area and so on are all contained with in this value $C_D$.

To use it a numerical model, we can define a new function that calculates this formula for us with `v` being passed in as an argument.

In [ ]:
def baseball_drag(v):
    """
    A paramerization for the air drag on a baseball dependent only on the speed v
    
    Returns the drag coefficient per unit mass (C_D) so that the drag force per unit mass 
    can be computed as
    
    Fdrag/m = C_D * v**2
    """
    
    vd = 35 # m/s
    Δ = 5 # m/s
    CD = 0.0039 + 0.0058 / (1 + np.exp(v-vd) / Δ)
    return CD

Python code can be written using Unicode (not just ASCII letters and numbers).  The capital delta, $\Delta$, could also be replaced with the word `Delta` (and it often is in other programming languages).

To type this character (assuming you do not have a Greek keyboard) Jupyter has a neat feature that if you type
    
`\Delta<TAB>`

in a code cell, it is automatically turned into a Δ symbol. 

A wonderful property of using numpy functions like `np.exp` is that it can handle both individual numbers,

In [ ]:
print(baseball_drag(30))

or evaluate an entire array in with one function call

In [ ]:
v = np.arange(20, 60, 10)
print(baseball_drag(v))

Let's decrease the stepsize for `v` and plot $B_2/m$ vs $v$.

In [ ]:
v = np.arange(20, 60, 1)
fig, axes = plt.subplots()
plt.plot(v, baseball_drag(v))
plt.xlabel("$v$ (m/s)")
plt.ylabel("$C_D$  (m$^{-1}$)")
plt.show()

With this parameterization, we can now calculate the drag forces per unit mass, broken up in to component vectors, as 

$$\begin{align}
\frac{F_{drag,x}}{m} = - C_D v v_x \\
\frac{F_{drag,y}}{m} = - C_D v v_y \\
\frac{F_{drag,z}}{m} = - C_D v v_z
\end{align}
$$

A typical value for a 149 gram baseball is $C_D = 6 \times 10^{-3}$ (Adair, 1990).

## Effects of spin

We've seen air drag can considerable change the range of a projectile - and is true for both cannon shells and baseballs. In addition, it turns out that *spin* can radically affect the trajectory of projectile. This is called the Magnus effect or Magnus Force.



### Magnus Force

* [What Is The Magnus Force?](https://www.youtube.com/watch?v=23f1jvGUWJs)



In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("23f1jvGUWJs")

Let's model this so-called Magnus force and add it in to our computational model for projectile motion.  Consider the drag acting on either side of a spinning ball rotating at a angular velocity $\omega$.

<a title="Rdurkacz [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Sketch_of_Magnus_effect_with_streamlines_and_turbulent_wake.svg"><img width="512" alt="Sketch of Magnus effect with streamlines and turbulent wake" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Sketch_of_Magnus_effect_with_streamlines_and_turbulent_wake.svg/512px-Sketch_of_Magnus_effect_with_streamlines_and_turbulent_wake.svg.png"></a>

We know that $F_{drag}$ depends on the relative velocity of the ball with the air.  When the ball is spinning, one side is moving faster and one side is moving slower relative to the air. Hence, the drag forces on each side are not the same.

The details of calculating the Magnus force directly is beyond the scope of this course (requires Fluid Dynamics) but we can understand the functional form of the Magnus force.

The drag force is proportional to the square of the relative velocity. On the one side this is  $(v + r \omega))^2$ while on the other it is $(v - r\omega)^2$.  The Magnus force is proportional to the difference between these two terms:

$$ F_m \propto (v+r\omega)^2 - (v-r\omega)^2 \sim v r \omega\;.$$

So the next spin-dependent force should have a functional form like

$$ \frac{F_M}{m}= C_M \omega v_x $$

For a thrown baseball, we can assume that the velocity is predominantly in the $x$-direction. So the Magnus force will act in direction perpendicular to both the $x$-direction and the axis of rotation.   

In the fully general case, we would need to consider a cross-product like 

$$\frac{\vec{F}_M}{m} = C_M \vec{\omega} \times \vec{v}$$

for the Magnus effect.

The coefficient $C_M$ is an experimentally determined parameter.  For a $m=149$ g baseball, *Adair (1990)* quotes a value of $C_M \approx 4.1 \times 10^{-4}$. 

Another way to understand the Magnus effect is how it was explained in the video. With a spinning ball, the air is deflected around the ball. So there is a force on the air by the ball. By Newton's third law there is an equal yet opposite force on the ball by the air. 

<a title="MatSouffNC858s [CC BY-SA 4.0 (https://creativecommons.org/licenses/by-sa/4.0)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Magnus-anim-canette.gif"><img width="512" alt="Magnus-anim-canette" src="https://upload.wikimedia.org/wikipedia/commons/f/f0/Magnus-anim-canette.gif"></a>

To add the effect of the Magnus force into our numerical solution to investigate the dynamics of a curve ball we need to think about our trajectory in all three dimensions.  

The total force on the baseball is given by

$$ \frac{\vec{F}}{m} = \vec{g} - C_D |\vec{v}| \vec{v} + C_M \left( \vec{\omega} \times \vec{v} \right) $$

The complete equations of motion are


\begin{align}
\frac{d x}{dt} &= v_x \\
\frac{d y}{dt} &= v_y \\
\frac{d z}{dt} &= v_z \\
\frac{d v_x}{dt} &= - C_D v v_x + C_M \left(\omega_y v_z - \omega_z v_y\right)\\
\frac{d v_y}{dt} &= - C_D v v_y + C_M \left(\omega_z v_x - \omega_x v_z\right) \\ 
\frac{d v_z}{dt} &= - C_D v v_z + C_M \left(\omega_x v_y - \omega_y v_x\right) - g
\end{align}

Here, the positive $z$-axis is taken as the vertical direction (up) direction.



## Code with rotation

Using Euler's method once again gives us the following code. Air drag forces and the Magnus forces are included all three directions.

In [ ]:
def BaseballStepper(dt=0.01, 
                        x0=0, y0=0, z0=3,
                        vx0=10, vy0=0, vz0=0,
                        ωx=0, ωy=0, ωz=0,
                        tmax=4):
    """
    Solve for the motion of a spinning baseball with air drag using Euler's method
    
    x, y, z are the positions
    vx, vy, vz are the velocity components
    ωx, ωy, ωz are components of the angular velocity vector
    """
    
    # initialize the model
    t = 0
    x = x0
    y = y0
    z = z0
    vx = vx0
    vy = vy0
    vz = vz0
    
    while True:
        model = {'t': t, 
                 'x': x, 'y': y, 'z': z,
                 'vx': vx, 'vy': vy, 'vz': vz}
        yield model # return the model state back to the caller
    
        if t > tmax:
            break
        if z < 0: # stop if the ball hits the ground
            break
            
        # drag forces (per unit mass)
        v = np.sqrt(vx**2+vy**2+vz**2)
        # drag coefficient depends on speed
        CD = baseball_drag(v)
        Fdrag_x = -CD * v * vx
        Fdrag_y = -CD * v * vy
        Fdrag_z = -CD * v * vz
        
        # Magnus forces (per unit mass)
        CM = 4.1e-4
        Fmagnus_x = CM * (ωy*vz - ωz*vy)
        Fmagnus_y = CM * (ωz*vx - ωx*vz)
        Fmagnus_z = CM * (ωx*vy - ωy*vx)
        
        # net force (per unit mass)
        g = 9.81
        Fnet_x = Fdrag_x + Fmagnus_x
        Fnet_y = Fdrag_y + Fmagnus_y
        Fnet_z = Fdrag_z + Fmagnus_z - g
        
        # use the Euler algorithm to update the state of the model
        x = x + vx*dt
        y = y + vy*dt
        z = z + vz*dt
        vx = vx + Fnet_x * dt
        vy = vy + Fnet_y * dt
        vz = vz + Fnet_z * dt
        t = t + dt

In [ ]:
def BaseballApp(ωx=0, ωy=0, ωz=0):
    
    # initial position
    x0 = 0
    y0 = 0
    z0 = 2 # m
    
    # initial velocity
    angle0 = 0 # degrees
    θ0 = np.deg2rad(angle0) # radians
    v0 = 50 # m/s (about 110 mph)
    
    vx0 = v0 * np.cos(θ0)
    vy0 = 0
    vz0 = v0 * np.sin(θ0)
    
    tmax = 10 # s
        
    # set up the model
    model = BaseballStepper(x0=x0, y0=y0, z0=z0,
                                vx0=vx0, vy0=vy0, vz0=vz0,
                                ωx=ωx, ωy=ωy, ωz=ωz,
                                tmax=tmax)
    
    # iterate the model
    ball = pd.DataFrame(model)

    # plot the results
    BaseballPlot(ball)

    return ball

In [ ]:
def BaseballPlot(ball):
    
    fig, axs = plt.subplots(1,3, figsize=(16, 4))
    axs[0].plot(ball.x, ball.z)
    axs[0].set_xlabel('x (m)')
    axs[0].set_ylabel('z (m)')
    axs[0].axvline(0, color='r')
    axs[0].axhline(0, color='k')
    axs[0].set_title('Side View')
        
    axs[1].plot(ball.x, ball.y)
    axs[1].set_xlabel('x (m)')
    axs[1].set_ylabel('y (m)')
    axs[1].axvline(0, color='r')
    axs[1].axhline(0, color='b')
    axs[1].set_title('Top Down View')
    
    axs[2].plot(ball.y, ball.z)
    axs[2].set_xlabel('y (m)')
    axs[2].set_ylabel('z (m)')
    axs[2].axvline(0, color='b')
    axs[2].axhline(0, color='k')
    axs[2].set_title('Front View')
    


Investigate the behaviour of throwing a baseball with different amounts of spin alng different axes.

In [ ]:
interactive(BaseballApp, 
            ωx=(-1000, 1000, 20),
            ωy=(-1000, 1000, 20),
            ωz=(-1000, 1000, 20)
           )

### 3D Plotting

In [ ]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D

def Baseball3DPlotApp(ωx=0, ωy=0, ωz=0):
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')

    ball = BaseballApp(ωx=ωx, ωy=ωy, ωz=ωz)
    ax.plot(ball.x, ball.y, ball.z, color='b')
    ax.plot(ball.x, ball.y, zs=0, zdir='z', color='k')
    ax.plot(ball.y, ball.z, zs=0, zdir='x', color='k')
    
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('z (m)')
    
interactive(Baseball3DPlotApp, 
            ωx=(-1000, 1000, 20),
            ωy=(-1000, 1000, 20),
            ωz=(-1000, 1000, 20)
           )

YouTube video:

* [Surprising Applications of the Magnus Effect](https://www.youtube.com/watch?v=2OSrvzNW9FE )

In [ ]:
YouTubeVideo("2OSrvzNW9FE")

- - -
### Textbook readings

This lecture is related CSM Chapter 3:

- 3.10 Visualizing Three-Dimensional Motion

but you may ignore the material on the "Open Source Physics 3D drawing framework" and focus on the discussion of the Magnus effect that starts on the middle of page 73. 

### DataCamp exercises

To learn more about Python, work through the following [DataCamp](http://datacamp.com) chapters:
- Introduction to Python: Python Basics
- Introduction to Python: Functions and Packages 
- Introduction to Python: NumPy 
- Intermediate Python for Data Science: Matplotlib 
- Intermediate Python for Data Science: Logic, Control Flow and Filtering 
- Intermediate Python for Data Science: Loops

These tutorials remains available until the end of the course.

### Assignment 1

Results...

### Assignment 2

You should be working on Assignment 2 now (due Sunday Sept. 29).  Those two problems are taken directly from the textbook, so reviewing the textbook could be useful. Also, you probably want to start from the code we developed in the corresponding lectures.

Please let me know if you want to meet to discuss this assignment via Zoom either 1-on-1 or with a group.

### Lab 1

Results...

### Lab 2

Please ask for help! Especially from me!

- - -